In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path


audio_file_noise = Path(
    "/lustre/scwpod02/client/kyutai/vaclav/data/tts_longform_debug/2/outputs/wikibooks_en/ft_4min/en_speaker_1_text_24.wav"
)
audio_file_drift = Path(
    "/lustre/scwpod02/client/kyutai/vaclav/data/tts_longform_debug/2/outputs/wikibooks_en/16s_context/en_speaker_3_text_14.wav"
)
audio_file = audio_file_noise

In [ ]:
import safetensors.torch


frames = safetensors.torch.load_file(audio_file.with_suffix(".safetensors"))["frames"]

In [24]:
from moshi.models.loaders import CheckpointInfo
from moshi.models.tts import TTSModel, DEFAULT_DSM_TTS_REPO, MimiModel
import sphn

In [25]:
checkpoint_info = CheckpointInfo.from_hf_repo(DEFAULT_DSM_TTS_REPO)
mimi = checkpoint_info.get_mimi(device="cpu")

In [33]:
audio, _sr = sphn.read(audio_file, sample_rate=24000)

In [34]:
import torch

with torch.no_grad():
    latents = mimi.encode_to_latent(torch.Tensor(audio[None, :, :]))

In [ ]:
from sklearn.neighbors import NearestNeighbors

frames_per_sec = 12.5
n_samples = int(frames_per_sec * 30)

nn = NearestNeighbors()
nn.fit(latents[0, :, :n_samples].detach().numpy().T)
nearest_distances = nn.kneighbors(latents[0, :, :].detach().numpy().T, n_neighbors=5)[0]
nearest_distances = nearest_distances.mean(axis=1)
nearest_distances /= nearest_distances[:n_samples].mean()

In [45]:
latents[0, :, :n_samples].detach().numpy().T.shape

(375, 512)

In [42]:
import plotly.express as px

In [43]:
px.line(nearest_distances)

In [ ]:
nearest_distances

array([1.0180777 , 0.78408537, 0.59579179, ..., 3.92966922, 3.93168974,
       3.78738104], shape=(5825,))

In [52]:
str(audio_file_drift.with_suffix(".drift.json"))

'/lustre/scwpod02/client/kyutai/vaclav/data/tts_longform_debug/2/outputs/wikibooks_en/16s_context/en_speaker_3_text_14.drift.json'

In [7]:
from tts_longeval.drift import get_drift_metrics


dm = get_drift_metrics(
    audio_file_noise,
    [0.25, 0.5, 0.75, 1.0],
)
dm

{'drift': 3.206189226473772,
 'd_q0.25': 1.5197134648867472,
 'd_q0.5': 3.5904545444656524,
 'd_q0.75': 3.83647472993677,
 'd_q1.0': 3.877652996434243}

In [49]:
dm